<a href="https://colab.research.google.com/github/AdrianAlbrecht/Collab/blob/main/WS_zad_3_Adrian_Albrecht_155376.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
from sklearn import datasets
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import distance_metrics
from sklearn.model_selection import train_test_split
import pandas as pd
import sklearn.metrics as metrics_sklearn

# Wczytywanie danych wine i iris

In [57]:
iris = datasets.load_iris()
wine = datasets.load_wine()
x_iris = iris.data
y_iris = iris.target
x_wine = wine.data
y_wine = wine.target

# Tworzymy egzemplarze Standard Scaler dla Iris i Wine

In [58]:
standarizer_iris = StandardScaler()
iris_features_stanarized = standarizer_iris.fit_transform(x_iris)
standarizer_wine = StandardScaler()
wine_features_stanarized = standarizer_wine.fit_transform(x_wine)

# Łączymy ustandaryzowane X i Y

In [59]:
iris = pd.DataFrame(iris_features_stanarized)
iris['class'] = y_iris
wine = pd.DataFrame(wine_features_stanarized)
wine['class'] = y_wine

## Dzielimy dane na testowe i treningowe

---

Ustalamy random_state=0 , aby wyniki zawsze był te same

In [60]:
iris_tst, iris_trn = train_test_split(iris, test_size=0.2, random_state= 0)
wine_tst, wine_trn = train_test_split(wine, test_size=0.2, random_state= 0)
iris_tst_x = iris_tst.iloc[:,:-1]
iris_tst_y = iris_tst.iloc[:,-1]
iris_trn_x = iris_trn.iloc[:,:-1]
iris_trn_y = iris_trn.iloc[:,-1]
wine_tst_x = wine_tst.iloc[:,:-1]
wine_tst_y = wine_tst.iloc[:,-1]
wine_trn_x = wine_trn.iloc[:,:-1]
wine_trn_y = wine_trn.iloc[:,-1]

# Tworzymy klasyfikatory dla Iris i Wine, dla ilości sąsiadów od 1 do 5, oraz dla wszystkich metryk zaimplementowanych w metodę


---
## Najpierw tworzymy spis wszystkich metryk



In [61]:
metrics = list(distance_metrics().keys())
metrics

['cityblock',
 'cosine',
 'euclidean',
 'haversine',
 'l2',
 'l1',
 'manhattan',
 'precomputed',
 'nan_euclidean']

# Usuwamy 'haversine', bo ten jest tylko na dwie cechy oraz 'precomputed', bo ten źle wpółpracuje z ujemnymi danymi

In [62]:
metrics.remove('haversine')
metrics.remove('precomputed')
len(metrics)

7

## Teraz tworzymy listę klasyfikatorów i pustą listę na predykcje klasyfikatorów

In [63]:
classifiers = []
predicts_iris = []
predicts_wine = []
for n in range(1,6):
  for metric in metrics:
    classifiers.append(KNeighborsClassifier(n_neighbors=n, metric=metric))
for classifier in classifiers:
  k_nn = classifier.fit(iris_trn_x, iris_trn_y)
  predicts_iris.append(k_nn.predict(iris_tst_x))
  k_nn = classifier.fit(wine_trn_x, wine_trn_y)
  predicts_wine.append(k_nn.predict(wine_tst_x))

# Porównamy predykcję dla każdego klasifikatora na bazie statystyki "accuracy"

In [67]:
acc_iris = []
acc_wine = []
for predicts in predicts_iris:
  acc_iris.append(metrics_sklearn.accuracy_score(iris_tst_y, predicts))
for predicts in predicts_wine:
  acc_wine.append(metrics_sklearn.accuracy_score(wine_tst_y, predicts))

# Porównajmy wartości

In [71]:
print(f'Dla iris: {acc_iris}')
print(f'Dla wine: {acc_wine}')

Dla iris: [0.9083333333333333, 0.85, 0.9083333333333333, 0.9083333333333333, 0.9083333333333333, 0.9083333333333333, 0.9083333333333333, 0.7583333333333333, 0.675, 0.7416666666666667, 0.7416666666666667, 0.7583333333333333, 0.7583333333333333, 0.7416666666666667, 0.85, 0.7166666666666667, 0.8, 0.8, 0.85, 0.85, 0.8, 0.7583333333333333, 0.7, 0.7166666666666667, 0.7166666666666667, 0.7583333333333333, 0.7583333333333333, 0.7166666666666667, 0.8416666666666667, 0.7916666666666666, 0.7916666666666666, 0.7916666666666666, 0.8416666666666667, 0.8416666666666667, 0.7916666666666666]
Dla wine: [0.9507042253521126, 0.9366197183098591, 0.9225352112676056, 0.9225352112676056, 0.9507042253521126, 0.9507042253521126, 0.9225352112676056, 0.9154929577464789, 0.9014084507042254, 0.8873239436619719, 0.8873239436619719, 0.9154929577464789, 0.9154929577464789, 0.8873239436619719, 0.9788732394366197, 0.9436619718309859, 0.9436619718309859, 0.9436619718309859, 0.9788732394366197, 0.9788732394366197, 0.94366

# Widzimy, że wartości się powtarzają, szczególnie te najwyższe (a te nas interesują), wyszukajmy zatem indeksy tych wartości i wypiszmy najlepsze klasyfikatory względem dokładności ("accuracy")

In [73]:
best_iris_acc = [i for i in range(len(acc_iris)) if acc_iris[i] == max(acc_iris)]
best_wine_acc = [i for i in range(len(acc_wine)) if acc_wine[i] == max(acc_wine)]
print(f'Dla iris: {best_iris_acc}')
print(f'Dla wine: {best_wine_acc}')

Dla iris: [0, 2, 3, 4, 5, 6]
Dla wine: [14, 18, 19]


In [75]:
print(f'Najlepsze klasyfikatory dla Iris z dokładnością {max(acc_iris)}:')
for i in best_iris_acc:
  print(classifiers[i])

Najlepsze klasyfikatory dla Iris z dokładnością 0.9083333333333333:
KNeighborsClassifier(metric='cityblock', n_neighbors=1)
KNeighborsClassifier(metric='euclidean', n_neighbors=1)
KNeighborsClassifier(metric='l2', n_neighbors=1)
KNeighborsClassifier(metric='l1', n_neighbors=1)
KNeighborsClassifier(metric='manhattan', n_neighbors=1)
KNeighborsClassifier(metric='nan_euclidean', n_neighbors=1)


In [76]:
print(f'Najlepsze klasyfikatory dla Wine z dokładnością {max(acc_wine)}:')
for i in best_wine_acc:
  print(classifiers[i])

Najlepsze klasyfikatory dla Wine z dokładnością 0.9788732394366197:
KNeighborsClassifier(metric='cityblock', n_neighbors=3)
KNeighborsClassifier(metric='l1', n_neighbors=3)
KNeighborsClassifier(metric='manhattan', n_neighbors=3)
